In [ ]:
## So first let's import the data and set up our libraries 
import pandas as pd 
wm_data = pd.read_excel("Weights-Measures_Data.xlsx")

In [3]:
##Great now let's take a peek under the hood of the data 
wm_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12755 entries, 0 to 12754
Data columns (total 35 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   Certificate of Inspection      12755 non-null  object        
 1   Inspection Number              12755 non-null  object        
 2   Date of Occurance              12755 non-null  datetime64[ns]
 3   Year (added)                   12755 non-null  int64         
 4   Business Unique ID             12750 non-null  object        
 5   Business Name                  12755 non-null  object        
 6   DBA Trade Name                 3552 non-null   object        
 7   Business Category              12755 non-null  object        
 8   Inspection type                12755 non-null  object        
 9   Device Category                12755 non-null  object        
 10  Device Class                   12699 non-null  object        
 11  Device Type    

In [4]:
## Okay...so now let's tackle these 7 key investigative questions: 
## 1. Identify the repeat offenders: businesses with multiple failed inspections or confiscated/condemned devices.
## 2. Identify the geographic hotspots of device violations by borough and postal code 
## 3. Identify the most commonly inspected and/or condemned device type 
## 4. Are there seasonal spikes or trends in failures or enforcement
## 5. What business types are most at risk of violations 
## 6. Which borough and business types received the highest confiscations
## 7. How likely are devices to be condemned based on routine vs. complaint-driven inspections.

In [21]:
## Let's first identify the repeat offenders: businesses with multiple failed inspections or confiscated/condemned devices.
## Let's first filter for failed inspections
## Then we'll count the number of failed inspections and look at the top 10

failed_inspections = wm_data[wm_data["Devices Condemned/Confiscated"] > 0]
repeat_offenders = failed_inspections["Business Name"].value_counts()
repeat_offenders.head(10)

Business Name
BOLLA OPERATING CORP.              85
SPEEDWAY LLC                       46
AMG RETAIL I LLC                   16
SPRAGUE OPERATING RESOURCES LLC    13
JAG TRANSPORTING INC               12
LOGIC FUEL TRANSPORTATION CORP      8
STAR ENERGY TRANSPORTATION INC.     7
E.S.F. TRANSPORT INC.               6
BERIL CORP                          5
FIVE BORO FUEL OIL LTD              5
Name: count, dtype: int64

In [6]:
# Now from our list of top repeat offenders, let's see what business category they fall into
# First, extract the list of top repeat offenders
top_offenders = [
    "BOLLA OPERATING CORP.",
    "SPEEDWAY LLC",
    "AMG RETAIL I LLC",
    "SPRAGUE OPERATING RESOURCES LLC",
    "JAG TRANSPORTING INC",
    "LOGIC FUEL TRANSPORTATION CORP",
    "STAR ENERGY TRANSPORTATION INC.",
    "E.S.F. TRANSPORT INC.",
    "BERIL CORP",
    "FIVE BORO FUEL OIL LTD"
]

# Filter the failed inspections for just these businesses
top_failed = failed_inspections[failed_inspections["Business Name"].isin(top_offenders)]

# Check their business categories
top_failed[["Business Name", "Business Category"]].drop_duplicates()


,Business Name,Business Category
456,SPRAGUE OPERATING RESOURCES LLC,Fuel Oil Dealer - 814
541,BOLLA OPERATING CORP.,Gas Station-Retail - 815
3140,SPEEDWAY LLC,Gas Station-Retail - 815
5234,BERIL CORP,Gas Station-Retail - 815
5253,STAR ENERGY TRANSPORTATION INC.,Fuel Oil Dealer - 814
5337,AMG RETAIL I LLC,Gas Station-Retail - 815
5689,JAG TRANSPORTING INC,Fuel Oil Dealer - 814
5754,LOGIC FUEL TRANSPORTATION CORP,Fuel Oil Dealer - 814
5944,SPRAGUE OPERATING RESOURCES LLC,Gasoline Truck-Retail - 822
6063,E.S.F. TRANSPORT INC.,Fuel Oil Dealer - 814


In [27]:
## Now lets get a ranked list of what devices were being condemned at each repeat offender's inspection
top_failed["Device Type"].value_counts()

Device Type
Octane Sample          140
Petrol Meter Type B     49
Petrol Pump Diesel       5
Petrol Pump Blend        4
No Fee Gas Pump          2
Petrol Meter Type A      2
Petrol Pump Single       1
Name: count, dtype: int64

In [29]:
##Now let's see which device type had the most failure per business
top_failed.groupby(["Business Name", "Device Type"]).size().sort_values(ascending=False)

Business Name                    Device Type        
BOLLA OPERATING CORP.            Octane Sample          82
SPEEDWAY LLC                     Octane Sample          38
AMG RETAIL I LLC                 Octane Sample          16
SPRAGUE OPERATING RESOURCES LLC  Petrol Meter Type B    13
JAG TRANSPORTING INC             Petrol Meter Type B    12
LOGIC FUEL TRANSPORTATION CORP   Petrol Meter Type B     7
STAR ENERGY TRANSPORTATION INC.  Petrol Meter Type B     7
E.S.F. TRANSPORT INC.            Petrol Meter Type B     6
FIVE BORO FUEL OIL LTD           Petrol Meter Type B     4
BERIL CORP                       Octane Sample           4
SPEEDWAY LLC                     Petrol Pump Diesel      3
                                 No Fee Gas Pump         2
BOLLA OPERATING CORP.            Petrol Pump Blend       2
SPEEDWAY LLC                     Petrol Pump Blend       2
BOLLA OPERATING CORP.            Petrol Pump Diesel      1
FIVE BORO FUEL OIL LTD           Petrol Meter Type A     1
LOG

Between 2023 and 2025, Bolla Operating Corp. topped the list of repeat offenders in NYC’s Weights and Measures inspections, failing 85 inspections across multiple locations — more than any other company. Speedway LLC followed with 46 failed inspections, and AMG Retail I LLC with 16.

The violations uncovered go beyond just faulty gas pumps. At the heart of these inspections were two critical issues: fuel quality and metering accuracy — both of which directly impact how much fuel consumers receive and what they pay for.

Bolla alone failed 82 octane sample tests, the highest in the city, suggesting a pattern of mislabeled or tampered fuel — a practice that could mean consumers are paying premium prices for regular-grade gas. In total, the top 10 offenders were cited in over 140 octane sampling failures, pointing to a widespread and systemic issue in fuel integrity across NYC.

At the same time, wholesalers like Sprague Operating Resources and JAG Transporting Inc. were repeatedly cited for Petrol Meter Type B violations — truck-mounted flow meters used to deliver heating oil or diesel to buildings and commercial customers. These meters, when miscalibrated, can result in underdelivery or overbilling, raising serious concerns about fuel delivery fraud in the supply chain.

In short, NYC’s most frequently cited fuel businesses failed not only to dispense fuel accurately — they often failed to deliver what they claimed was in the tank at all. These violations reflect more than broken hardware; they point to systemic negligence — or worse, economic deception — affecting everyday consumers and businesses alike.